In [1]:
# Importing all the libraries which are required
import numpy as np
import pickle
import cv2
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from tensorflow.keras.utils import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
# Initializing all the variables and default values for the program
EPOCHS = 25
INIT_LR = 1e-3
BS = 32
default_image_size = tuple((256, 256))
image_size = 0
directory_root = 'input/'
width=256
height=256
depth=3

In [3]:
# Function to create image into array
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, default_image_size)   
            return img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None


In [4]:
image_list, label_list = [], []
try:
    # Loading the images in the model
    root_dir = listdir(directory_root)

    for plant_folder in root_dir :
        plant_disease_folder_list = listdir(f"{directory_root}/{plant_folder}")
        # Processing images of each folder 
        
        for plant_disease_folder in plant_disease_folder_list:
            print(f"-> Processing {plant_disease_folder} ...")
            plant_disease_image_list = listdir(f"{directory_root}/{plant_folder}/{plant_disease_folder}/")

            for image in plant_disease_image_list[:200]:
                image_directory = f"{directory_root}/{plant_folder}/{plant_disease_folder}/{image}"
                if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
                    image_list.append(convert_image_to_array(image_directory))
                    label_list.append(plant_disease_folder)
    print("-> Image loading completed")  
except Exception as e:
    print(f"Error : {e}")

-> Processing Pepper__bell___Bacterial_spot ...
-> Processing Pepper__bell___healthy ...
-> Processing PlantVillage ...
-> Processing Potato___Early_blight ...
-> Processing Potato___healthy ...
-> Processing Potato___Late_blight ...
-> Processing Tomato_Bacterial_spot ...
-> Processing Tomato_Early_blight ...
-> Processing Tomato_healthy ...
-> Processing Tomato_Late_blight ...
-> Processing Tomato_Leaf_Mold ...
-> Processing Tomato_Septoria_leaf_spot ...
-> Processing Tomato_Spider_mites_Two_spotted_spider_mite ...
-> Processing Tomato__Target_Spot ...
-> Processing Tomato__Tomato_mosaic_virus ...
-> Processing Tomato__Tomato_YellowLeaf__Curl_Virus ...
-> Image loading completed


In [5]:
image_size = len(image_list)

In [6]:
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
pickle.dump(label_binarizer,open('label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)

In [7]:
print(label_binarizer.classes_)

['Pepper__bell___Bacterial_spot' 'Pepper__bell___healthy'
 'Potato___Early_blight' 'Potato___Late_blight' 'Potato___healthy'
 'Tomato_Bacterial_spot' 'Tomato_Early_blight' 'Tomato_Late_blight'
 'Tomato_Leaf_Mold' 'Tomato_Septoria_leaf_spot'
 'Tomato_Spider_mites_Two_spotted_spider_mite' 'Tomato__Target_Spot'
 'Tomato__Tomato_YellowLeaf__Curl_Virus' 'Tomato__Tomato_mosaic_virus'
 'Tomato_healthy']


In [8]:
np_image_list = np.array(image_list, dtype=np.float16) / 225.0

In [9]:
# Splitting data into train and test data
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.2, random_state = 42) 

In [10]:
aug = ImageDataGenerator(
    rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, 
    zoom_range=0.2,horizontal_flip=True, 
    fill_mode="nearest")

In [11]:
# Adding the parameters in the CNN model

model = Sequential()
inputShape = (height, width, depth)
chanDim = -1
if K.image_data_format() == "channels_first":
    inputShape = (depth, height, width)
    chanDim = 1
model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(n_classes))
model.add(Activation("softmax"))

In [12]:
opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / EPOCHS)
# distribution
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])

In [18]:
# Training model
history = model.fit(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // BS,
    epochs=EPOCHS, verbose=1
    )

Epoch 1/25
73/73 [==============================] - 199s 3s/step - loss: 0.0553 - accuracy: 0.8828 - val_loss: 0.1525 - val_accuracy: 0.6497
Epoch 2/25
73/73 [==============================] - 204s 3s/step - loss: 0.0462 - accuracy: 0.9047 - val_loss: 0.2041 - val_accuracy: 0.5533
Epoch 3/25
73/73 [==============================] - 202s 3s/step - loss: 0.0444 - accuracy: 0.9038 - val_loss: 0.5584 - val_accuracy: 0.3587
Epoch 4/25
73/73 [==============================] - 203s 3s/step - loss: 0.0433 - accuracy: 0.9120 - val_loss: 0.2519 - val_accuracy: 0.6227
Epoch 5/25
73/73 [==============================] - 203s 3s/step - loss: 0.0596 - accuracy: 0.8712 - val_loss: 0.1379 - val_accuracy: 0.7343
Epoch 6/25
73/73 [==============================] - 203s 3s/step - loss: 0.0472 - accuracy: 0.8974 - val_loss: 0.4276 - val_accuracy: 0.4755
Epoch 7/25
73/73 [==============================] - 202s 3s/step - loss: 0.0420 - accuracy: 0.9085 - val_loss: 0.1709 - val_accuracy: 0.7005
Epoch 8/25
73

In [14]:
from keras.models import load_model

In [19]:
# Saving the model
model.save('cnn_model.h5')

In [28]:
# Testing the model for an input image

loaded_model = load_model('cnn_model.h5')
image_dir="pepperbellhealthy.jpg"

im=convert_image_to_array(image_dir)
np_image_li = np.array(im, dtype=np.float16) / 225.0
npp_image = np.expand_dims(np_image_li, axis=0)

In [29]:
result=model.predict(npp_image)
itemindex = np.where(result==np.max(result))
print(label_binarizer.classes_[itemindex[1][0]])

1/1 [==============================] - 0s 220ms/step
Pepper__bell___healthy
